<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/Interference_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Interference using pretrained models**

In [1]:
import torch
from IPython.display import Image, clear_output
import os
import shutil

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#GDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Setup complete. Using torch 1.12.1+cu113 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)
Mounted at /content/drive


In [2]:
image_folder_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_newPatient_250px"

model_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/5-fold_20221019/Models/moblenetv3_large_100.pth"

In [10]:
# specify the image paths
image_list = [image_folder_path +"/スライド"+str(i+1)+".jpeg" for i in range(len(os.listdir(image_folder_path)))]
label_list = [0]*len(image_list)

###**Load model**

In [4]:
##########################
# Load model 
##########################
!pip install --quiet timm
import timm
import torch.nn as nn


model_ft = timm.create_model('mobilenetv3_large_100', pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 2)

#ネットワークの読み込み
model_ft.load_state_dict(torch.load(model_path))

     |████████████████████████████████| 548 kB 36.6 MB/s 
     |████████████████████████████████| 163 kB 62.8 MB/s 


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth" to /root/.cache/torch/hub/checkpoints/mobilenetv3_large_100_ra-f55367f5.pth


<All keys matched successfully>

In [11]:
import torch.utils.data as data
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

class SimpleImageDataset(Dataset):
    def __init__(self, img_list, label_list, transform):
        self.transform = transform
        self.img_list = img_list
        self.label_list = label_list
        self.item_dict = {}
        self.age = []
        #print(img_list)
        #print(label_list)

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        image_path = self.img_list[idx]
        pilr_image = Image.open(image_path).convert("RGB")
        tensor_image = self.transform(pilr_image)
        target = torch.tensor(self.label_list[idx])      
        return tensor_image, target

test_data_transforms = transforms.Compose([
                #Expand2square((0,0,0)),
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_dataset = SimpleImageDataset(image_list, label_list, test_data_transforms)

test_loader = DataLoader(image_list, batch_size = 16, shuffle = True, pin_memory=True, num_workers=0)


In [ ]:
from torchvision import datasets, models, transforms
from PIL import *

img_transforms = transforms.Compose([
                #Expand2square((0,0,0)),
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

def image_loader(image_path):
    """load image, returns cuda tensor"""
    image = Image.open(image_path)
    image = img_transforms(image).float()
    image = image.unsqueeze(0) 
    return image.cuda()

image_tensor = image_loader(image_list[0])

model_ft.to(device)
model_ft.eval()
output = model_ft(image_tensor)
_, pred = torch.max(output, 1) 
pred = pred.to('cpu').detach().numpy().copy()

prob = nn.Softmax(dim=1)(output) #calculate probalility
prob = prob[0][1].cpu().detach().numpy().copy() #probalility of being positive

print(pred) 
print(prob)



In [ ]:
for image_path in image_list:  
      #target = target.squeeze(1)     
      image_tensor = image_tensor.to(device)
      target = target.to(device)
      # forward pass: compute predicted outputs by passing inputs to the model
      output = model_ft(image_tensor)
      _, pred = torch.max(output, 1) 
    
      prob = nn.Softmax(dim=1)(output) #calculate probalility
      prob = prob[0][1].cpu().detach() #probalility of being positive
      print(prob)
      print(pred) 
      
      probs.append(prob) #予測確率
      preds.append(int(pred))  #予測結果
      targets.append(int(target)) #ラベル